In [2]:
!pip install gdown

In [3]:
!gdown --id 1Hs_p9uEEBOh3GpZGzMnOFBfSLsz8Bq1x
!gdown --id 1Ufxl1HEy1pvpL2xvrzOKWR8vd5Nu84vc

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Hs_p9uEEBOh3GpZGzMnOFBfSLsz8Bq1x
To: /kaggle/working/Difa_model2_coba_dulu_maseh.tflite
100%|██████████████████████████████████████| 14.2M/14.2M [00:00<00:00, 30.5MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ufxl1HEy1pvpL2xvrzOKWR8vd5Nu84vc
To: /kaggle/working/LABELS.txt
100%|█████████████████████████████████████████| 70.0/70.0 [00:00<00:00, 308kB/s]


In [4]:
!pip install tflite-support

  Obtaining dependency information for tflite-support from https://files.pythonhosted.org/packages/39/37/2602fa31e308e1f38897819e5ea41e69f4b1852197cdbbb3a0e8105ffbbe/tflite_support-0.4.4-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 19.9 MB/s eta 0:00:00:00:0100:01


 # ADDING METADATA TO MODEL

In [5]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb



_MODEL_PATH = "/kaggle/working/Difa_model2_coba_dulu_maseh.tflite"
_LABEL_FILE = "/kaggle/working/LABELS.txt"


""" ... """
"""Creates the metadata for an image classifier."""

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "DIFA MODEL image classifier"
model_meta.description = ("Identify hand-sign SIBI in the "
                          "image and other set of 24 Alphabet in SIBI hand-sign")
model_meta.version = "v1"
model_meta.author = "CH2-PS487"


In [6]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()


# ADDING THE INPUT METADATA INFO TO MODEL

In [7]:
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(224, 224))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# ADDING THE OUTPUT METADATA INFO TO MODEL

In [10]:
import os

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 24 labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0] * 24  # Modify based on your actual output range
output_stats.min = [0.0] * 24
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(_LABEL_FILE)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]


# ADDING SUBGRAPH INFO TO MODEL

In [11]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [12]:
populator = _metadata.MetadataPopulator.with_model_file(_MODEL_PATH)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([_LABEL_FILE])
populator.populate()

In [13]:
displayer = _metadata.MetadataDisplayer.with_model_file(_MODEL_PATH)
json_file = displayer.get_metadata_json()

print(json_file)

{
  "name": "DIFA MODEL image classifier",
  "description": "Identify hand-sign SIBI in the image and other set of 24 Alphabet in SIBI hand-sign",
  "version": "v1",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified. The expected image is 224 x 224, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single byte between 0 and 255.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
   

displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)
export_json_file = os.path.join(FLAGS.export_directory,
                                os.path.splitext(model_basename)[0] + ".json")
json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)